<a href="https://jupyterhub.user.eopf.eodc.eu/hub/login?next=%2Fhub%2Fspawn%3Fnext%3D%252Fhub%252Fuser-redirect%252Fgit-pull%253Frepo%253Dhttps%253A%252F%252Fgithub.com%252Feopf-toolkit%252Feopf-101%2526branch%253Dmain%2526urlpath%253Dlab%252Ftree%252Feopf-101%252F05_zarr_tools%252F51_eopf_stac_r.ipynb%23fancy-forms-config=%7B%22profile%22%3A%22choose-your-environment%22%2C%22image%22%3A%22unlisted_choice%22%2C%22image%3Aunlisted_choice%22%3A%224zm3809f.c1.de1.container-registry.ovh.net%2Feopf-toolkit-r%2Feopf-toolkit-r%3Alatest%22%2C%22autoStart%22%3A%22true%22%7D" target="_blank">
<button style="background-color:#0072ce; color:white; padding:0.6em 1.2em; font-size:1rem; border:none; border-radius:6px; margin-top:1em;">
🚀 Launch this notebook in JupyterLab
</button>
</a>

### Introduction

In this section, we will explore programmatic access of the EOPF Zarr Collections available in the [EOPF Sentinel Zarr Sample Service STAC Catalog](https://stac.browser.user.eopf.eodc.eu/) using R. We will introduce R packages that enable us to effectively access and search through STAC catalogs.

### What we will learn

- 🔍 How to <b>programmatically browse</b> through available collections available via the EOPF Zarr STAC Catalog, using the `rstac` package.
- 📊 Explore <b>collection metadata</b> in user-friendly terms
- 🎯 How to <b>search for specific data</b>

### Prerequisites

An R environment is required to follow this tutorial, with R version >= 4.4.0. For local development, we recommend using either [RStudio](https://posit.co/download/rstudio-desktop/) or [Positron](https://posit.co/products/ide/positron/) and making use of [RStudio projects](https://support.posit.co/hc/en-us/articles/200526207-Using-RStudio-Projects) for a self-contained coding environment.

The [`rstac` package](https://brazil-data-cube.github.io/rstac/) is required to follow this tutorial. You can install it directly from CRAN:

In [ ]:
# install.packages("rstac")

Then load the package into your environment:

In [ ]:
library(rstac)

## Connect to the EOPF Sample Service STAC API

To access the EOPF Sample Service STAC catalog in R, we need to give the URL of the STAC API source ([https://stac.core.eopf.eodc.eu/](https://stac.core.eopf.eodc.eu/)) using the function `stac()`.

The object `stac_source` is a query containing information used _to_ connect to the API, but it does not actually make any requests. To make requests to the API, we will always need to use `get_request()` or `put_request()`, as appropriate. Running `get_request()` on `stac_source` actually retrieves the catalogue:

In [ ]:
stac_source <- stac("https://stac.core.eopf.eodc.eu/")

stac_source |>
  get_request()

## Browse collections

A STAC _Collection_ exists to relate similar data sets together through space, time, and shared metadata. Each Sentinel mission and the downstream analysis-ready data are examples of STAC Collections. To browse STAC Collections, the `collections()` function is used. We can see that there are 12 collections available in the API:

In [ ]:
stac_collections <- stac_source |>
  collections() |>
  get_request()

stac_collections

The default printing of the `stac_collections()` object summarises what's been returned, but does not give all of the information. To see more about what's been returned, we use `str()`.

In [ ]:
stac_collections |>
  str(max.level = 1)

Here, we can see that there is an entry `"collections"` within `stac_collections`, which we access to return the collections themselves (using `head()` to only return a few). This shows additional details about each collection, such as the collection id, title, description, and additional fields in the collections.

In [ ]:
stac_collections[["collections"]] |>
  head(3)

The Sentinel-2 Level-2A collection can be accessed by getting the first entry in `stac_collections()[["collections"]]`

In [ ]:
stac_collections[["collections"]][[1]]

However, the best way to access a specific collection is to search for it directly using the collection ID. The ID, "sentinel-2-l2a", is visible in the Collection output above. It is also accessible in the browsable STAC catalog of the EOPF Sentinel Zarr Samples Service, on the [page for that collection](https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a), under "Source."

![](../img/51_eopf-stac-access-collections-id.png)

The collection ID can be supplied directly in the `collections()` function. If we look at the query without getting the result, we can see that it has been formed using the `collection_id`, "sentinel-2-l2a", as a filter parameter.

In [ ]:
sentinel_2_l2a_query <- stac_source |>
  collections(collection_id = "sentinel-2-l2a")

sentinel_2_l2a_query

And that running `get_request()` will return the collection itself:

In [ ]:
sentinel_2_l2a_query |>
  get_request()

## Browse items

Within collections, there are _items_. Items are the building blocks for STAC. At their core, they are GeoJSON data, along with additional metadata which ensures data provenance is maintained and specific data attributes are captured. A single capture from a Sentinel mission is an example of a STAC item. To get an overview of items within a collection, the `items()` function is used.

An important thing to note with `rstac` is that you cannot continue to build queries on top of ones that have already had their results returned (via `get_request()`). It may make sense for a typical workflow in R to "get" the collection, then to try to get the items from it, but this will produce an error:

In [ ]:
# sentinel_2_l2a_collection <- stac_source |>
#   collections(collection_id = "sentinel-2-l2a") |>
#   get_request()
# 
# sentinel_2_l2a_collection |>
#   items()

    Error: Invalid rstac_query value.

If you see this error --- `"Invalid rstac_query value"` --- ensure that you are running `get_request()` at the very end of your query building functions. Using `items()` this way, we can see that it returns a summary of the collection's items:

In [ ]:
sentinel_2_l2a_collection_items <- stac_source |>
  collections(collection_id = "sentinel-2-l2a") |>
  items() |>
  get_request()

sentinel_2_l2a_collection_items

The first 10 items are returned. This number can be changed via the `limit` argument in `items()`

In [ ]:
stac_source |>
  collections(collection_id = "sentinel-2-l2a") |>
  items(limit = 20) |>
  get_request()

### Item properties

We can look closer at individual items to see the metadata attached to them. Items are stored under `"features"`:

In [ ]:
sentinel_2_l2a_collection_items[["features"]] |>
  head(2)

And an individual item contains a lot of information, such as its bounding box:

In [ ]:
sentinel_2_l2a_first_item <- sentinel_2_l2a_collection_items[["features"]][[1]]

sentinel_2_l2a_first_item[["bbox"]]

And many more additional properties, with their properties under `"properties"` in an individual item.

In [ ]:
sentinel_2_l2a_first_item[["properties"]] |>
  names()

For example, the EOPF instrument mode:

In [ ]:
sentinel_2_l2a_first_item[["properties"]][["eopf:instrument_mode"]]

For the rest of the tutorial, we will use a small helper function that accesses a given property for the _first_ item returned in a search.

In [ ]:
get_first_item_property <- function(search_results, property) {
  search_results[["features"]][[1]][["properties"]][[property]]
}

sentinel_2_l2a_collection_items |>
  get_first_item_property("eopf:instrument_mode")

### Search for items

If the goal is to access data from a specific mission, it is best to _search_ within a collection's items, using some of the properties explored above. It's possible to search based on a number of criteria, including a bounding box, time frame, and other mission properties.

#### Search by a bounding box

To narrow down items based on a bounding box or time frame, the `stac_search()` function is used. The collection ID is provided in the `collections()` argument, and bounding box and time frame are `bbox` and `datetime`, respectively.

The bounding box values take the sequence of: minimum longitude, minimum latitude, maximum longitude, and maximum latitude, and their coordinate reference system is WGS84. For items whose bounding boxes intersect with Vienna:

In [ ]:
stac_source |>
  stac_search(
    collections = "sentinel-2-l2a",
    bbox = c(16.1736, 48.1157, 16.5897, 48.3254)
  ) |>
  get_request()

This does -- again by default -- return the first 10 items, but the number returned can be increased via the `limit` argument in `stac_search()`.

#### Search by a time frame

When searching for a specific time frame, items that have a datetime property that _intersects_ with the given time frame will be returned. It's therefore best to search for a closed or open interval, rather than a specific date and time (which might be difficult to match exactly to an item's time!). The date-time must be given in RFC 3339 format.

To search for a closed interval, separate two date-times by a "/", e.g. `"2024-12-01T01:00:00Z/2024-12-01T05:00:00Z"`:

In [ ]:
matching_timeframe_items <- stac_source |>
  stac_search(
    collections = "sentinel-2-l2a",
    datetime = "2024-12-01T01:00:00Z/2024-12-01T05:00:00Z"
  ) |>
  get_request()

matching_timeframe_items

We can access the matching item's `datetime` property to see that it falls within the specified interval:

In [ ]:
matching_timeframe_items |>
  get_first_item_property("datetime")

To search by an open interval, ".." is used to indicate the open end, e.g. `"../2024-01-01T23:00:00Z"` representing prior to that date-time, and `"2024-01-01T23:00:00Z/.."` representing after it:

In [ ]:
stac_source |>
  stac_search(
    collections = "sentinel-2-l2a",
    datetime = "2025-01-01T23:00:00Z/.."
  ) |>
  get_request()

#### Search by other item properties

As shown above, there are a number of other properties attached to STAC items. We can also search using these properties. The `stac_search()` function is limited to properties like bounding box and time frame, so instead we use `ext_filter()`. This is a function that makes use of the Common Query Language (CQL2) filter extension, and allows us to do more complicated searching and querying using SQL-like language. It is also important to note that when using `ext_filter()`, we switch to using `post_request()` instead of `get_request()`.

For this searching, it is helpful to know the _data type_ for an item property in advance, as this will impact what operation to use within `ext_filter()`. We create an additional helper function for this:

In [ ]:
get_item_property_type <- function(property = NULL) {
  api_res <- rstac:::make_get_request("https://stac.core.eopf.eodc.eu/api") |>
    rstac:::content_response_json()
  item_properties_schema <- api_res[["components"]][["schemas"]][["ItemProperties"]][["properties"]]

  property_types <- lapply(item_properties_schema, function(x) {
    x[["anyOf"]][[1]][["type"]]
  })

  if (is.null(property)) {
    property_types
  } else {
    property_types[[property]]
  }
}

When no argument is passed to this function, it will return all of the properties and their types:

In [ ]:
get_item_property_type()

When the name of a property is passed, it will return the type of that property. We can see, for example, that `platform` is a string, while `instruments` is an array.

In [ ]:
get_item_property_type("platform")
get_item_property_type("instruments")

Since `platform` is a string, we use `==` to indicate equality. For example, to search for items whose platform is "sentinel-2b":

In [ ]:
sentinel_2b_platform_results <- stac_source |>
  stac_search(collections = "sentinel-2-l2a") |>
  ext_filter(platform == "sentinel-2b") |>
  post_request()

sentinel_2b_platform_results

sentinel_2b_platform_results |>
  get_first_item_property("platform")

If the search value is contained in another variable, the variable must be escaped in the search by using double curly braces:

In [ ]:
search_platform <- "sentinel-2b"

sentinel_2b_platform_results <- stac_source |>
  stac_search(collections = "sentinel-2-l2a") |>
  ext_filter(platform == {{ search_platform }}) |>
  post_request()

sentinel_2b_platform_results |>
  get_first_item_property("platform")

Note also that there is no `limit` argument in `ext_filter()`. To limit the number of items returned, the limit is supplied in `stac_search()` beforehand, since these search functions build upon one another:

In [ ]:
stac_source |>
  stac_search(collections = "sentinel-2-l2a", limit = 1) |>
  ext_filter(platform == {{ search_platform }}) |>
  post_request()

To search for items with cloud cover of less than 40, we use `<=`:

In [ ]:
stac_source |>
  stac_search(collections = "sentinel-2-l2a") |>
  ext_filter(`eo:cloud_cover` <= 40) |>
  post_request() |>
  get_first_item_property("eo:cloud_cover")

If we want to search for items where `instruments` is "msi", we use the `a_contains()` function. We need to use this instead of `==` because `instruments` is an `array`, as seen above. This means it operates like a list within R, and can contain multiple values -- `a_contains()` searches for the value `"msi"` within the list of values that is `instruments`.

In [ ]:
stac_source |>
  stac_search(collections = "sentinel-2-l2a") |>
  ext_filter(a_contains(instruments, "msi")) |>
  post_request() |>
  get_first_item_property("instruments")

Note that there is currently a bug with how the `rstac` package converts the API's data to an R object. This bug makes it unclear that `instruments` is a _list_ that needs to be searched within (instead of a single value). There is an [issue to fix this bug in the `rstac` github repository](https://github.com/brazil-data-cube/rstac/issues/175). We hope that the helper function `get_item_property_type()` will be helpful in the meantime to determine which filtering operation to use.

The [documentation for `ext_filter()`](https://brazil-data-cube.github.io/rstac/reference/ext_filter.html#details) contains information on how to construct many more searches than we've shown here.

#### Combine search criteria

You can combine multiple filter criteria by specifying them together. We have already seen how to combine multiple criteria (collection ID and bounding box, for example) in `stac_search()` by using the named arguments. We can also filter by bounding box and datetime in the same way. Multiple criteria in `ext_filter()` are separated by `&&`:

In [ ]:
multiple_criteria_items <- stac_source |>
  stac_search(
    collections = "sentinel-2-l2a",
    bbox = c(16.1736, 48.1157, 16.5897, 48.3254),
    datetime = "../2025-06-01T23:00:00Z"
  ) |>
  ext_filter(
    platform == "sentinel-2a" &&
      `eo:cloud_cover` <= 40
  ) |>
  post_request()

multiple_criteria_items |>
  get_first_item_property("datetime")

multiple_criteria_items |>
  get_first_item_property("platform")

multiple_criteria_items |>
  get_first_item_property("eo:cloud_cover")

## Browse assets

Finally, assets fall under STAC items and direct users to the actual data itself. Each asset refers to data associated with the Item that can be downloaded or streamed.

We will look at the assets for a specific item from the Sentinel-2 Level-2A collection. Like collections, items can be filtered by their IDs. Their IDs are also available through the API:

In [ ]:
sentinel_2_l2a_collection_items[["features"]][[1]]

Or through the STAC catalog of the EOPF Sentinel Zarr Samples Service, on the page for that item, under "Source":

![](../img/51_eopf-stac-access-items-id.png)

To select a single item, supply its ID in the `items()` function:

In [ ]:
example_item <- stac_source |>
  collections("sentinel-2-l2a") |>
  items(sentinel_2_l2a_collection_items[["features"]][[1]][["id"]]) |>
  get_request()

example_item

There are a number of helpful functions for working with an item's assets, such as `items_assets()` which lists them:

In [ ]:
# List the assets in an item
example_item |>
  items_assets()

And `assets_select()` which allows us to select specific assets (in this case, the "Surface Reflectance - 10m" asset):

In [ ]:
sr_10m <- example_item |>
  assets_select(asset_names = "SR_10m")

sr_10m

For example, the "product" asset will be useful to working with EOPF Sample Service Zarr data, as this is the top-level Zarr hierarchy. We can select this asset, and then use `assets_url()` to get its URL:

In [ ]:
example_item |>
  assets_select(asset_names = "product") |>
  assets_url()

It is also helpful to know which assets actually contain Zarr data. Assets can be _Zarr groups_, which share common dimensions and coordinates, and contain Zarr arrays within them. An asset can also be an individual Zarr array.

To look more at this, we will extract metadata attached to the Zarr assets. The `"assets"` entry of `example_item` contains a lot of useful information, but it is a bit difficult to read and manipulate:

In [ ]:
names(example_item[["assets"]])

example_item[["assets"]][["SR_10m"]]

So, we will reformat it to be easier to work with. To do so, we first load the `tidyverse` package for data manipulation (installing it first, if necessary):

In [ ]:
# install.packages("tidyverse")
library(tidyverse)

We will retain only the title and roles of each asset.

In [ ]:
asset_metadata <- example_item[["assets"]] |>
  map(\(asset) {
    asset[c("title", "roles")]
  })

head(asset_metadata, 5)

Then, we can filter to only keep assets who have the `roles` "dataset" (these are Zarr groups):

In [ ]:
asset_metadata |>
  keep(\(asset) {
    "dataset" %in% asset[["roles"]]
  })

Or to those who have `roles` "data", but _not_ "dataset" or "metadata" (these are individual Zarr arrays):

In [ ]:
zarr_arrays <- asset_metadata |>
  keep(\(asset) {
    "data" %in% asset[["roles"]] &
      !("dataset" %in% asset[["roles"]] | "metadata" %in% asset[["roles"]])
  })

names(zarr_arrays)

head(zarr_arrays, 3)

## 💪 Now it is your turn

The following expercises will help you master the STAC API and understand how to find the data you need.

### Task 1: Explore Your Area of Interest

- Go to [http://bboxfinder.com/](http://bboxfinder.com/) and select an area of interest (AOI) (e.g. your hometown, a research site, etc.)
- Copy the bounding box coordinates of your area of interest
- Change the provided code above to search for data over your AOI

### Task 2: Temporal Analysis

- Compare data availability across different years for the <b>Sentinel-2 L-2A Collection</b>.
- Search for items in the year 2022
- Repeat the search for the year 2024

### Task 3: Explore the SAR Mission and combine multiple criteria

- Do the same for a different `Collection`, the <b>Sentinel-1 Level-1 GRD</b>, e.g. you can use the ID `sentinel-1-l1-grd`
- How many assets are available for the year 2024?

## Conclusion

This tutorial has provided a clear and practical introduction on how you can programmatically access and search through EOPF Sentinel Zarr Sample Service STAC API using R.

## What's next?

In the following [section](./52_eopf_stac_qgis.qmd), we will provide a glimpse of the actual status on how to integrate the **Sentinel Products** of the [EOPF Sentinel Zarr Sample Service STAC Catalog](https://stac.browser.user.eopf.eodc.eu/?.language=en) in **Qgis**